In [83]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
from itertools import islice, chain
import hashlib
import os
import math
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp
from simulation import Simulation

In [91]:
variables_all = {
    'min_tests_daily_proportion': (0.001, 0.01),
    'max_tests_daily_proportion': (0.02, 0.1),
    'desired_positive_proportion': (0.01, 0.5),
}

problem = {
    'num_vars': len(variables_all),
    'names': variables_all.keys(),
    'bounds': [list(x) for x in variables_all.values()]
}

In [93]:
def compute_row_hash(row):
    return hashlib.sha1(np.array(row)).hexdigest()

def generate_samples_df(variables, sampling_fun):
    problem = build_problem(variables)
    samples = sampling_fun(problem)
    
    df = pd.DataFrame(samples, columns=variables.keys())
    
    df['hash'] = df.apply(compute_row_hash, axis=1)
    df.set_index('hash', inplace=True)
    return df

def spit_params_dicts(samples_df):
    for idx, row in df.iterrows():
        params = dict(row)
        params['run_id'] = idx
        yield params

In [8]:
def load_results_if_present(result_id):
    result_file = f'results/r_{result_id}.csv'
    return pd.read_csv(result_file, index_col=0) if os.path.isfile(result_file) else None

In [98]:
pdf = generate_samples_df(variables_all, lambda x: ms.sample(x, 100, seed=1234))
pdf

,min_tests_daily_proportion,max_tests_daily_proportion,desired_positive_proportion
hash,,,
391599cb126342b2118da600ab2bd21943269728,0.010,0.100000,0.500000
bb940f32474486292190a20bcebb862b2eda6a2e,0.004,0.100000,0.500000
0ea26d6f201c03c401a4248fb8a5300e45e066e8,0.004,0.046667,0.500000
7bcfc1ef2e2db066e8151beab503bcb05c162702,0.004,0.046667,0.173333
f6e3cee02eaeb4de00689579e57582a7a78fd0e5,0.007,0.020000,0.336667
...,...,...,...
033b75c5c627d7e0ed7e0de3b9673a618b4f450b,0.010,0.046667,0.336667
0ea26d6f201c03c401a4248fb8a5300e45e066e8,0.004,0.046667,0.500000
e65329d39a5704644cd5486a06890e7f595e2d54,0.010,0.046667,0.500000


In [16]:
for idx, row in tqdm(pdf.iterrows()):
    results = load_results_if_present(idx)
    if results is None:
        param_dict = row.to_dict()
        param_dict['quiet'] = True
        param_dict['visualize'] = False
        param_dict['verbose'] = False
        param_dict['print_sum'] = False
        param_dict['run_id'] = idx

        sim = Simulation(**param_dict)
        sim.run()

        results = load_results_if_present(idx)
        assert results is not None

In [78]:
def compute_metrics(exp):
    metrics = []
    for rid in exp1:
        rdf = load_results_if_present(rid)
        fatalities = rdf['fatalities'].iloc[-1]
        unaffected = rdf['susceptible'].iloc[-1]
        sev3 = rdf['severity2'].iloc[-1]
        metrics.append((fatalities, unaffected, sev3))
    return np.array(metrics, dtype=np.float64)